In [12]:
# CELL 1: SETUP
# This cell installs the necessary Python libraries and configures the API client.

!pip install groq openai -q
print("✅ Successfully installed groq and openai libraries.")

import os
import json
from openai import OpenAI
from getpass import getpass

try:
    api_key = os.environ["GROQ_API_KEY"]
    print("✅ Groq API Key found in environment variables.")
except KeyError:
    api_key = getpass("🔑 Please enter your Groq API Key: ")

os.environ["GROQ_API_KEY"] = api_key

try:
    client = OpenAI(
        api_key=os.environ.get("GROQ_API_KEY"),
        base_url="https://api.groq.com/openai/v1",
    )
    print("✅ Groq client initialized successfully and ready to use.")
except Exception as e:
    print(f"❌ Error initializing Groq client: {e}")
    client = None

✅ Successfully installed groq and openai libraries.
✅ Groq API Key found in environment variables.
✅ Groq client initialized successfully and ready to use.


In [13]:
# CELL 2: TASK 1 - CONVERSATION MANAGER
# This cell defines the class that will manage our conversation history.
# Running this cell will not produce any output, but it makes the class
# available for the next cell to use.

class ConversationManager:
    """
    Manages conversation history with robust summarization and truncation.
    """
    def __init__(self, client, model="llama-3.1-70b-versatile", k_summarize=3):
        self.client = client
        self.model = model
        self.history = [{"role": "system", "content": "You are a helpful assistant."}]
        self.k_summarize = k_summarize  # Summarize after every k-th run
        self.turn_count = 0

    def add_message(self, role, content):
        """Adds a message and triggers summarization if the condition is met."""
        self.history.append({"role": role, "content": content})
        if role == "user":
            self.turn_count += 1

        # Check if it's time to summarize after a user's turn
        if self.turn_count > 0 and self.turn_count % self.k_summarize == 0:
            print(f"\n--- ⚠️ Triggering summarization after {self.turn_count} user turns. ---")
            self._summarize()

    def _summarize(self):
        """Internal method to summarize the history using the Groq API."""
        # We don't summarize the very last message to maintain immediate context.
        history_to_summarize = self.history[:-1]

        if len(history_to_summarize) <= 1: # Only a system message
            print("--- Not enough history to summarize. ---")
            return

        prompt = (
            "Summarize the following conversation concisely. Capture key points, user intent, "
            "and any important details mentioned. The summary will be used as context for a "
            "new conversation. Do not add introductory phrases like 'The summary is...'.\n\n"
            f"{json.dumps(history_to_summarize)}"
        )

        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.1,
            )
            summary = response.choices[0].message.content.strip()
            last_message = self.history[-1]

            # Replace old history with a new system prompt containing the summary
            self.history = [
                {"role": "system", "content": f"This is a summary of the previous conversation: {summary}"},
                last_message
            ]
            print("--- ✅ Conversation summarized successfully. ---")
        except Exception as e:
            print(f"--- ❌ Error during summarization: {e} ---")

    def truncate_by_turns(self, n_turns):
        """Keeps only the last n conversation turns (1 turn = 1 user + 1 assistant)."""
        num_messages = n_turns * 2
        # Always keep the system message + the last n messages
        self.history = [self.history[0]] + self.history[-num_messages:]
        print(f"\n--- History truncated to the last {n_turns} turns. ---")

    def truncate_by_length(self, max_chars):
        """Keeps recent messages that fit within a specified character limit."""
        current_chars = 0
        new_history = []
        # Iterate backwards from the end, keeping messages until the limit is reached
        for message in reversed(self.history[1:]): # Skip system message
            current_chars += len(message["content"])
            if current_chars > max_chars:
                break
            new_history.insert(0, message)

        self.history = [self.history[0]] + new_history
        print(f"\n--- History truncated to fit under {max_chars} characters. ---")

    def print_history(self):
        """Prints the current history in a readable format."""
        print("\n" + "="*60)
        print("📜 CURRENT CONVERSATION HISTORY 📜")
        print("="*60)
        for msg in self.history:
            print(f"[{msg['role'].upper()}]: {msg['content']}")
        print("="*60)

In [14]:
# CELL 3: TASK 1 - DEMONSTRATION
# This cell demonstrates all functionalities of the ConversationManager class.

if client:
    print("🚀 STARTING DEMONSTRATION FOR TASK 1: CONVERSATION MANAGEMENT 🚀")

    print("\n\n--- 1. Testing Periodic Summarization (triggers on 3rd user message) ---")
    convo_manager = ConversationManager(client, k_summarize=3)

    print("\n--- Turn 1 ---")
    convo_manager.add_message("user", "Hi, I need help planning a trip to Japan.")
    convo_manager.add_message("assistant", "Of course! Japan is amazing. What cities are you interested in?")
    convo_manager.print_history()

    print("\n--- Turn 2 ---")
    convo_manager.add_message("user", "I want to see Tokyo for the city life and Kyoto for the culture.")
    convo_manager.add_message("assistant", "Great choices! How long will your trip be?")
    convo_manager.print_history()

    print("\n--- Turn 3 (Summarization should be triggered automatically after this) ---")
    convo_manager.add_message("user", "I'm planning for about 12 days in late October.")
    convo_manager.print_history()

    print("\n\n--- 2. Testing Truncation by Number of Turns ---")
    trunc_manager_turns = ConversationManager(client)
    trunc_manager_turns.add_message("user", "Message 1"); trunc_manager_turns.add_message("assistant", "Response 1")
    trunc_manager_turns.add_message("user", "Message 2"); trunc_manager_turns.add_message("assistant", "Response 2")
    trunc_manager_turns.add_message("user", "Message 3"); trunc_manager_turns.add_message("assistant", "Response 3")
    print("\n--- Full History Before Truncation ---")
    trunc_manager_turns.print_history()
    trunc_manager_turns.truncate_by_turns(n_turns=1)
    trunc_manager_turns.print_history()

    print("\n\n--- 3. Testing Truncation by Character Length ---")
    trunc_manager_len = ConversationManager(client)
    trunc_manager_len.add_message("user", "This is a very long initial user message designed specifically to test the character truncation feature.")
    trunc_manager_len.add_message("assistant", "This is an equally verbose assistant response to add more length to the conversation history.")
    trunc_manager_len.add_message("user", "This is a shorter, more recent message.")
    trunc_manager_len.add_message("assistant", "A short reply.")
    print("\n--- Full History Before Truncation ---")
    trunc_manager_len.print_history()
    trunc_manager_len.truncate_by_length(max_chars=100)
    trunc_manager_len.print_history()
else:
    print("Client not initialized. Please run Cell 1 and provide your API key.")

🚀 STARTING DEMONSTRATION FOR TASK 1: CONVERSATION MANAGEMENT 🚀


--- 1. Testing Periodic Summarization (triggers on 3rd user message) ---

--- Turn 1 ---

📜 CURRENT CONVERSATION HISTORY 📜
[SYSTEM]: You are a helpful assistant.
[USER]: Hi, I need help planning a trip to Japan.
[ASSISTANT]: Of course! Japan is amazing. What cities are you interested in?

--- Turn 2 ---

📜 CURRENT CONVERSATION HISTORY 📜
[SYSTEM]: You are a helpful assistant.
[USER]: Hi, I need help planning a trip to Japan.
[ASSISTANT]: Of course! Japan is amazing. What cities are you interested in?
[USER]: I want to see Tokyo for the city life and Kyoto for the culture.
[ASSISTANT]: Great choices! How long will your trip be?

--- Turn 3 (Summarization should be triggered automatically after this) ---

--- ⚠️ Triggering summarization after 3 user turns. ---
--- ❌ Error during summarization: Error code: 400 - {'error': {'message': 'The model `llama-3.1-70b-versatile` has been decommissioned and is no longer supported. Plea

In [15]:
# CELL 4: TASK 2 - JSON EXTRACTION LOGIC
# This cell defines the schema for data extraction and the function that
# calls the Groq API using its 'tool calling' feature. Running it will
# not produce output but makes the function ready for the next cell.

user_details_schema = {
    "type": "function",
    "function": {
        "name": "extract_user_info",
        "description": "Extracts user information from a chat message.",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {"type": "string", "description": "The user's full name."},
                "email": {"type": "string", "description": "The user's email address."},
                "phone": {"type": "string", "description": "The user's phone number."},
                "location": {"type": "string", "description": "The user's city or address."},
                "age": {"type": "integer", "description": "The user's age in years."},
            },
            "required": []
        }
    }
}

def extract_info_from_chat(chat_text, client, model="llama-3.1-70b-versatile"):
    """
    Uses Groq's tool calling feature to extract structured information.
    """
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are an expert information extraction system. Your task is to extract user details from the provided text and format it using the 'extract_user_info' tool. If a detail is not present, do not include it in the output."},
                {"role": "user", "content": chat_text}
            ],
            tools=[user_details_schema],
            tool_choice={"type": "function", "function": {"name": "extract_user_info"}},
            temperature=0.0,
        )

        tool_call = response.choices[0].message.tool_calls[0]
        arguments_str = tool_call.function.arguments

        extracted_data = json.loads(arguments_str)
        return extracted_data
    except (json.JSONDecodeError, IndexError, AttributeError) as e:
        return {"error": "Failed to extract or parse valid data.", "details": str(e)}
    except Exception as e:
        return {"error": "An unexpected API error occurred.", "details": str(e)}

In [16]:
# CELL 5: TASK 2 - DEMONSTRATION
# This cell runs the demonstration for the JSON extraction function,
# processing several sample chats and showing the structured output.

if client:
    print("🚀 STARTING DEMONSTRATION FOR TASK 2: JSON EXTRACTION 🚀")

    chat_samples = [
        "Hi, my name is John Doe. I'm 29 years old and I live in New York. You can reach me at john.doe@email.com or on my mobile at 555-123-4567.",
        "Please sign me up for the newsletter. My name is Jane Smith and my email is jane.s@web.com. My phone is 555-222-3333.",
        "hey can you help me? im david, my e-mail is dave.c@mail.net and my fone is 555-987-6543. i'm based in Chicago",
        "My phone number is 555-555-5555. My name is Alex Ray. I am 42 years old. My email address is alex.ray@work.org."
    ]

    for i, sample in enumerate(chat_samples):
        print(f"\n\n--- 📝 Processing Sample {i+1} ---")
        print(f"Input Text: \"{sample}\"")

        extracted_json = extract_info_from_chat(sample, client)

        print("\n✅ Extracted Information (Validated JSON):")
        print(json.dumps(extracted_json, indent=2))
else:
    print("Client not initialized. Please run Cell 1 and provide your API key.")

🚀 STARTING DEMONSTRATION FOR TASK 2: JSON EXTRACTION 🚀


--- 📝 Processing Sample 1 ---
Input Text: "Hi, my name is John Doe. I'm 29 years old and I live in New York. You can reach me at john.doe@email.com or on my mobile at 555-123-4567."

✅ Extracted Information (Validated JSON):
{
  "error": "An unexpected API error occurred.",
  "details": "Error code: 400 - {'error': {'message': 'The model `llama-3.1-70b-versatile` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}"
}


--- 📝 Processing Sample 2 ---
Input Text: "Please sign me up for the newsletter. My name is Jane Smith and my email is jane.s@web.com. My phone is 555-222-3333."

✅ Extracted Information (Validated JSON):
{
  "error": "An unexpected API error occurred.",
  "details": "Error code: 400 - {'error': {'message': 'The model `llama-3.1-70b-versat